In [ ]:
import scipy.io

mat = scipy.io.loadmat('../../data/synthetic/signal/rx_x/AWGN_8-PSK_1e+06_1_0.15_18.mat')


In [ ]:
s = mat['rx_x'][:,0]
s.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(s.real)
plt.plot(s.imag)

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
import sys
sys.path.append("../")
from kdmc.data.sbasic import get_sbasic_datasets

get_sbasic_datasets(f'../../data', 1024, 0, False)

In [ ]:
data = np.load(os.path.join("../../data/synthetic/signal", "sbasic.npz"))
iq = data['iq']
modulation = data['modulation']
yml = data['yml']
yml_nf = data['yml_nf']
y = data['y']
snr = data['snr']

In [ ]:
# Without tx and rx filters
(yml_nf.argmax(1) == y.argmax(1)).mean()

In [ ]:
# With "root" rx and tx filter
(yml.argmax(1) == y.argmax(1)).mean()

In [ ]:
from kdmc.plot import plot_acc_vs_snr, plot_acc_vs_snr_by_class

df = pd.DataFrame({'snr': snr, 'acc': (yml.argmax(1) == y.argmax(1)), 'y': y.argmax(1), 'y_ml': yml.argmax(1)})
df_snr = df.groupby('snr', as_index=False)['acc'].mean().sort_values('snr')
fig = plot_acc_vs_snr(df_snr.acc, df_snr.snr, title="Accuracy vs SNR")

In [ ]:
from kdmc.plot import plot_acc_vs_snr_by_class
from kdmc.data.sbasic import SBasic

df_snr_class = df.groupby(['snr', 'y'], as_index=False)['acc'].mean()
fig = plot_acc_vs_snr_by_class(df_snr_class, labels=SBasic.classes, title="Accuracy vs SNR")

In [ ]:
df = pd.DataFrame({'snr': snr, 'acc': (yml_nf.argmax(1) == y.argmax(1)), 'y': y.argmax(1), 'y_ml': yml_nf.argmax(1)})
df_snr = df.groupby('snr', as_index=False)['acc'].mean().sort_values('snr')
fig = plot_acc_vs_snr(df_snr.acc, df_snr.snr, title="Accuracy vs SNR")

In [ ]:
df_snr_class = df.groupby(['snr', 'y'], as_index=False)['acc'].mean()
fig = plot_acc_vs_snr_by_class(df_snr_class, labels=SBasic.classes, title="Accuracy vs SNR")

In [ ]:
import h5py
import pandas as pd

mods = []
snrs = []
Ls = []
Ms = []
keys = []
data = {}
with h5py.File('../../data/synthetic/1024/dataset1024.mat') as f:
    for key in f['ds'].keys():
        keys.append(key)
        splits = key.split('rer')
        mods.append(splits[0])
        snrs.append(splits[1])
        Ls.append(splits[2])
        Ms.append(splits[4])
        data[key] = f['ds'][key][:]


In [ ]:
df = pd.DataFrame({'mod': mods, 'snr': snrs, 'L': Ls, 'M': Ms, 'key': keys})
df['L'] = df['L'].astype(int)
df['M'] = df['M'].astype(int)
df['snr'] = df.snr.apply(lambda x: x.replace('neg', '-'))
df['snr'] = df['snr'].astype(float)
df.to_csv('../../data/synthetic/1024/dataset1024_keys.csv', index=False)

In [ ]:
sps8_keys = df.loc[(df.L == 8) & (df.M == 1), 'key'].values
list_data = []
for key in sps8_keys:
    list_data.append(data[key][:,:2])

In [ ]:
import numpy as np

dataset = np.concatenate(list_data, axis=2)
dataset = dataset.reshape(dataset.shape[2], dataset.shape[1], dataset.shape[0])

In [ ]:
dataset.shape